# 03_TRAIN_CONTINUAL — Entrenamiento Continual con *presets*

**Qué hace este notebook:**

Este notebook entrena y evalúa modelos en **aprendizaje continual** usando una configuración unificada desde `configs/presets.yaml`.  
Permite: (1) lanzar un *run* base, (2) comparar métodos continual con idéntica configuración de datos/modelo, y (3) generar un **resumen** y **agregados** de resultados.

---

## 🎯 Objetivos
- Centralizar la configuración (modelo, datos, optimizador, continual) vía `presets.yaml`.
- Entrenar con **H5 offline** o **CSV+runtime encode** (auto-detectado).
- Comparar métodos (`naive`, `ewc`, `rehearsal`, `rehearsal+ewc`) de forma reproducible.
- Exportar resúmenes a `outputs/summary/continual_summary_agg.csv`.

## ✅ Prerrequisitos
- `data/processed/tasks.json` o `data/processed/tasks_balanced.json` generados.
- Si usas **offline** (H5), haberlos creado con `tools/encode_tasks.py`.
- Revisar `configs/presets.yaml` (secciones: `model`, `data`, `optim`, `continual`).

## ⚠️ Notas importantes
- No combines `use_offline_spikes=True` con `encode_runtime=True`.
- La **semilla** del experimento viene de `CFG["data"]["seed"]` (reproducibilidad).
- El nombre de la carpeta de salida codifica preset, método y meta (ver `src/runner.py`).

<a id="toc"></a>

## 🧭 Índice

- [1) Setup del entorno y paths](#sec-01)
- [2) Carga del preset unificado (`configs/presets.yaml`)](#sec-02)
- [3) Verificación de datos y selección de `tasks.json`](#sec-03)
- [4) Factories DataLoaders + Modelo (+ tasks)](#sec-04)
- [5) (Opcional) Parche: imprimir *it/s* por época](#sec-06)
- [6) Ejecución base con el preset (eco de config + run)](#sec-07)
- [7) Comparativa de métodos (mismo preset/semilla/datos)](#sec-08)
- [8) Barrido de combinaciones (opcional)](#sec-09)
- [9) Resumen completo: inventario → parseo → agregados → tabla](#sec-10)



<a id="sec-01"></a>

## 1) Setup del entorno y paths

**Objetivo:** preparar el entorno de ejecución con granularidad de hilos, selección de dispositivo y rutas del proyecto.

- Fija variables de entorno para limitar hilos BLAS (reproducibilidad y evitar oversubscription).
- Detecta `ROOT` (raíz del repo) y lo añade a `sys.path`.
- Importa utilidades del proyecto (datasets, modelos, presets).
- Selecciona dispositivo (`cuda` si está disponible).
- Activa optimizaciones de PyTorch (TF32/cuDNN) para acelerar entrenamiento en GPU.

> **Nota:** No se leen presets aquí todavía; únicamente se configura el runtime global.

[↑ Volver al índice](#toc)

In [1]:
# =============================================================================
# Imports y setup de entorno (threads, paths, dispositivo)
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

# Raíz del repo y sys.path
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Librerías del proyecto
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model
from src.utils import load_preset

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ajustes de rendimiento (opcional)
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("Device:", device)

Device: cuda


<a id="sec-02"></a>

## 2) Carga del preset unificado (`configs/presets.yaml`)

**Objetivo:** cargar un *preset* y derivar toda la configuración de trabajo.

Contenido:
- Lee el preset (`PRESET = "fast" | "std" | "accurate"`).
- Construye el `ImageTransform` según el modelo del preset.
- Extrae parámetros de **datos/codificación temporal** (`ENCODER`, `T`, `GAIN`, `SEED`).
- Extrae configuración del **DataLoader** (workers, prefetch, pin/persistent).
- Prepara *augment* opcional (`AUG_CFG`) y **balanceo online** (si procede).
- Guardarraíl: prohíbe usar a la vez `use_offline_spikes=True` y `encode_runtime=True`.

> **Consejo:** cambia el valor de `PRESET` aquí para barrer configuraciones sin tocar código en más sitios.

[↑ Volver al índice](#toc)

In [2]:
# =============================================================================
# Config global: presets.yaml
# =============================================================================
from pathlib import Path
from src.datasets import ImageTransform, AugmentConfig
from src.utils import load_preset

PRESET = "rate_t16_rgb"  # fast | std | accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# ---- Modelo / Transform ------------------------------------------------------
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"],
    CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]),
    crop_top=None
)

# ---- Datos / codificación temporal ------------------------------------------
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])

USE_OFFLINE_SPIKES = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE     = bool(CFG["data"].get("encode_runtime", False))

# ---- DataLoader / augment / balanceo ----------------------------------------
NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)           # robusto ante None
PREFETCH    = int(CFG["data"].get("prefetch_factor") or 2)       # <- casteo robusto
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
          if CFG["data"].get("aug_train") else None

USE_ONLINE_BALANCING = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or 50)
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)

# Guardarraíles
if USE_OFFLINE_SPIKES and RUNTIME_ENCODE:
    raise RuntimeError("Config inválida: use_offline_spikes=True y encode_runtime=True a la vez.")

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BALANCING} bins={BAL_BINS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")

[PRESET=rate_t16_rgb] model=pilotnet_snn 200x66 gray=False
[DATA] encoder=rate T=16 gain=0.5 seed=42
[LOADER] workers=8 prefetch=2 pin=True persistent=True
[BALANCE] online=False bins=50
[RUNTIME_ENCODE] False | [OFFLINE_SPIKES] True


<a id="sec-03"></a>

## 3) Verificación de datasets y selección de `tasks.json`

Comprueba que existen los CSV de `train/val/test` por tarea, y (si corresponde)
que `train_balanced.csv` está disponible para el modo **offline balanceado**.

- Lee `tasks_balanced.json` si `USE_OFFLINE_BALANCED=True`; si faltan, cae a `tasks.json`.
- Construye `task_list` con rutas por split.

> *Salida esperada:* listado de tareas y su CSV de `train`. Mensaje de OK/aviso.

[↑ Volver al índice](#toc)

In [3]:
# =============================================================================
# Verificación de datos (splits y, si procede, H5)
# =============================================================================
from pathlib import Path as _P
import json

PROC = ROOT / "data" / "processed"

# --- Elegir tasks según el preset ---
USE_BALANCED = bool(CFG.get("prep_offline", {}).get("use_balanced_tasks", False))
tb_name = (CFG.get("prep", {}).get("tasks_balanced_file_name") or "tasks_balanced.json")
t_name  = (CFG.get("prep", {}).get("tasks_file_name")           or "tasks.json")

cand_bal = PROC / tb_name
cand_std = PROC / t_name
TASKS_FILE = cand_bal if (USE_BALANCED and cand_bal.exists()) else cand_std

with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]
print("Usando:", TASKS_FILE.name)
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# Guardarraíl: si se pidió balanced, exigir train_balanced.csv
if USE_BALANCED:
    for t in task_list:
        train_path = _P(tasks_json["splits"][t["name"]]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en modo balanced, pero encontré '{train_path.name}'."
            )

# Si entrenas con H5 offline, comprueba que existan
if USE_OFFLINE_SPIKES:
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []
    for t in task_list:
        run = t["name"]
        base = PROC / run
        for split in ("train", "val", "test"):
            expected = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not expected.exists():
                missing.append(str(expected))
    if missing:
        print("[WARN] Faltan H5 compatibles con el preset.")
        print("       Genera primero con 02_ENCODE_OFFLINE.ipynb (o tools/encode_tasks.py).")

print("OK: verificación de splits.")
print(f"Preset en uso: {PRESET}")


Usando: tasks.json
 - circuito1: train.csv
 - circuito2: train.csv
OK: verificación de splits.
Preset en uso: rate_t16_rgb


<a id="sec-04"></a>

## 4) Factories unificados: DataLoaders + Modelo (+ tasks)

Construye en **una sola pasada** todos los componentes a partir del `CFG`:

- `build_components_for(CFG, ROOT)` → `tfm`, `make_loader_fn`, `make_model_fn`  
  - El **loader** respeta automáticamente los flags del preset (H5 offline vs. CSV+encode runtime, workers, prefetch, pin_memory, persistent, augmentación y balanceo online).
  - El **modelo** se crea según `model.name` y aplica, cuando procede, hiperparámetros de neurona.
- `build_task_list_for(CFG, ROOT)` → `task_list`, `tasks_file`  
  - Selecciona `tasks_balanced.json` si el preset lo pide y existe; en caso contrario usa `tasks.json`.

[↑ Volver al índice](#toc)


In [4]:
# === Factories y task list coherentes con el PRESET cargado ===
from src.utils import build_task_list_for, build_components_for

# Construye tfm, make_loader_fn y make_model_fn leyendo TODO de CFG (igual que hacías a mano):
tfm, make_loader_fn, make_model_fn = build_components_for(CFG, ROOT)

# Elige automáticamente tasks_balanced.json si el preset lo pide y existe; si no, tasks.json
task_list, tasks_file = build_task_list_for(CFG, ROOT)

print("Tasks file:", tasks_file.name)
print("make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).")


Tasks file: tasks.json
make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).


<a id="sec-05"></a>

## 5) (Opcional) Parche: imprimir iteraciones/segundo por época

Sobrescribe temporalmente `training.train_supervised` para:

- Medir **it/s** por época (útil para benchmarks de rendimiento).
- Mantener el resto del entrenamiento sin cambios funcionales.

> Para restaurar el comportamiento original: `training.train_supervised = orig_train_supervised`.

[↑ Volver al índice](#toc)


In [5]:
# =============================================================================
# (Opcional) Parche: it/s + Early Stopping (controlado por preset)
# =============================================================================
import time, json
from pathlib import Path
import torch
from torch import nn, optim
from torch.amp import autocast, GradScaler

import src.training as training
from src.utils import set_seeds

orig_train_supervised = training.train_supervised  # backup

def train_supervised_ips_es(model: nn.Module, train_loader, val_loader, loss_fn: nn.Module,
                            cfg, out_dir: Path, method=None):
    """
    it/s + Early Stopping:
      - Activo si cfg.es_patience y cfg.es_min_delta no son None.
      - Criterio: min val_loss con tolerancia es_min_delta.
    Escribe manifest.json con 'history' y 'early_stop_epoch' (si aplica).
    """
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    if cfg.seed is not None:
        set_seeds(cfg.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    opt = optim.Adam(model.parameters(), lr=cfg.lr)
    use_amp = bool(cfg.amp and torch.cuda.is_available())
    scaler = GradScaler(enabled=use_amp)

    # ES params (leídos del preset)
    patience = getattr(cfg, "es_patience", None)
    min_delta = getattr(cfg, "es_min_delta", None)
    use_es = (patience is not None) and (min_delta is not None)

    best_val = float("inf")
    wait = 0
    early_stop_epoch = None

    history = {"train_loss": [], "val_loss": []}
    for epoch in range(1, cfg.epochs + 1):
        # -------- train --------
        model.train()
        running = 0.0; nb = 0
        t0 = time.perf_counter()

        for x, y in train_loader:
            x = training._permute_if_needed(x).to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            opt.zero_grad(set_to_none=True)
            with autocast("cuda", enabled=use_amp):
                y_hat = model(x)
                loss = loss_fn(y_hat, y)
                if method is not None:
                    loss = loss + method.penalty()

            if use_amp:
                scaler.scale(loss).backward()
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(opt); scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                opt.step()

            running += loss.item(); nb += 1

        dt = time.perf_counter() - t0
        ips = nb / dt if dt > 0 else float("nan")
        print(f"[TRAIN it/s] epoch {epoch}/{cfg.epochs}: {ips:.1f} it/s  ({nb} iters en {dt:.2f}s)")
        train_loss = running / max(1, nb)

        # -------- val --------
        model.eval()
        v_running = 0.0; nvb = 0
        with torch.no_grad():
            for x, y in val_loader:
                x = training._permute_if_needed(x).to(device, non_blocking=True)
                y = y.to(device, non_blocking=True)
                with autocast("cuda", enabled=use_amp):
                    y_hat = model(x)
                    v_loss = loss_fn(y_hat, y)
                v_running += v_loss.item(); nvb += 1
        val_loss = v_running / max(1, nvb)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

        # -------- Early Stopping check --------
        if use_es:
            improved = (best_val - val_loss) > float(min_delta)
            if improved:
                best_val = val_loss
                wait = 0
            else:
                wait += 1
                if wait >= int(patience):
                    early_stop_epoch = epoch
                    print(f"[EarlyStopping] Stop en epoch={epoch} (best_val={best_val:.6f})")
                    break

    manifest = {
        "epochs": cfg.epochs, "batch_size": cfg.batch_size, "lr": cfg.lr,
        "amp": cfg.amp, "seed": cfg.seed, "history": history,
        "early_stop_epoch": early_stop_epoch,
    }
    (out_dir / "manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    return history

training.train_supervised = train_supervised_ips_es
print("Parche it/s + EarlyStopping ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised")


Parche it/s + EarlyStopping ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised


<a id="sec-06"></a>

## 6) Ejecución base con el preset (eco de config + run)

Lanza **un experimento** con el método y parámetros definidos en el preset (`CFG["continual"]`).  
Se imprimen los campos más relevantes (modelo, datos, loader) y se guarda la salida en `outputs/continual_*`.

[↑ Volver al índice](#toc)

In [6]:
# =============================================================================
# Ejecución base con el preset (eco de config + run)
# =============================================================================
from src.runner import run_continual

# Echo de configuración “resumido” (lo esencial para el run)
print(f"[RUN] preset={PRESET} | method={CFG['continual']['method']} "
      f"| seed={CFG['data']['seed']} | enc={CFG['data']['encoder']} "
      f"| kwargs={CFG['continual'].get('params', {})}")
print(f"[MODEL] {MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] T={CFG['data']['T']} gain={CFG['data']['gain']} "
      f"| offline_spikes={CFG['data']['use_offline_spikes']} "
      f"| runtime_encode={CFG['data']['encode_runtime']}")
print(f"[LOADER] workers={CFG['data']['num_workers']} "
      f"prefetch={CFG['data']['prefetch_factor']} pin={CFG['data']['pin_memory']} "
      f"persistent={CFG['data']['persistent_workers']} "
      f"| aug={bool(CFG['data']['aug_train'])} "
      f"| balance_online={CFG['data']['balance_online']}")

out_path, _ = run_continual(
    task_list=task_list,
    make_loader_fn=make_loader_fn,   # wrapper (Celda 4)
    make_model_fn=make_model_fn,     # factory (Celda 5)
    tfm=tfm,
    cfg=CFG,                         # preset completo
    preset_name=PRESET,              # solo naming
    out_root=ROOT / "outputs",
    verbose=True,
)
print("OK:", out_path)

[RUN] preset=rate_t16_rgb | method=ewc | seed=42 | enc=rate | kwargs={'lam': '1.0e9', 'fisher_batches': 1000}
[MODEL] pilotnet_snn 200x66 gray=False
[DATA] T=16 gain=0.5 | offline_spikes=True | runtime_encode=False
[LOADER] workers=8 prefetch=2 pin=True persistent=True | aug=True | balance_online=False

--- Tarea 1/2: circuito1 | preset=rate_t16_rgb | method=ewc_lam_1e+09 | B=64 T=16 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 5.2 it/s  (391 iters en 75.54s)
[TRAIN it/s] epoch 2/2: 5.6 it/s  (391 iters en 69.61s)

--- Tarea 2/2: circuito2 | preset=rate_t16_rgb | method=ewc_lam_1e+09 | B=64 T=16 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 5.3 it/s  (170 iters en 32.23s)
[TRAIN it/s] epoch 2/2: 5.7 it/s  (170 iters en 29.83s)
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_rate_t16_rgb_ewc_lam_1e+09_lam_1e+09_rate_t16_rgb_rate_model-PilotNetSNN_66x200_rgb_seed_42


<a id="sec-07"></a>

## 7) Comparativa de métodos (mismo preset / misma semilla / mismos datos)

Esta celda ejecuta varias corridas cambiando **únicamente** el método de aprendizaje
continual, manteniendo fijos el `preset` cargado en la Celda 2 (modelo, loader, AMP,
LR, epochs, T, gain, tamaño de imagen, etc.).

- Usa `CFG` tal cual (ya trae `continual.method`/`params`, `data.encoder`, `data.seed`, etc.).
- Para cada método, se clona `CFG` y se sobreescriben **solo** `continual.method` y `continual.params`.
- Los resultados se escriben en `outputs/continual_<preset>_<tag>_<encoder>_model-..._seed_<seed>/continual_results.json`.
- Después, usa las Celdas 10–13 para generar el resumen y el CSV de agregados.

**Requisitos**:
- Si usas *offline spikes*, asegúrate de que existen H5 compatibles con el preset:
  encoder/T/gain/size/to_gray. Si no, el loader emitirá un `FileNotFoundError`.

[↑ Volver al índice](#toc)

In [ ]:
# === COMPARATIVA DE MÉTODOS: mismo preset, misma semilla, mismos datos ===
from copy import deepcopy
from src.runner import run_continual
from src.utils import build_task_list_for, build_components_for

CFG_BASE = deepcopy(CFG)
METHODS = {
    "naive": {},
    "ewc": {"lam": 7e8, "fisher_batches": 800},
    "rehearsal": {"buffer_size": 3000, "replay_ratio": 0.1},
    "rehearsal+ewc": {"buffer_size": 3000, "replay_ratio": 0.1, "lam": 7e8, "fisher_batches": 800},
}

runs_out = []
for method_name, method_params in METHODS.items():
    cfg_i = deepcopy(CFG_BASE)
    cfg_i["continual"]["method"] = method_name
    cfg_i["continual"]["params"] = method_params
    if "rehearsal" in method_name:
        cfg_i["data"]["persistent_workers"] = False

    # (Re)construye factories por si el cfg cambia
    tfm_i, make_loader_fn_i, make_model_fn_i = build_components_for(cfg_i, ROOT)
    task_list_i, tasks_file_i = build_task_list_for(cfg_i, ROOT)

    print(f"\n=== RUN: preset={PRESET} | method={method_name} | seed={cfg_i['data']['seed']} "
          f"| enc={cfg_i['data']['encoder']} | kwargs={method_params} ===")
    out_dir, _ = run_continual(
        task_list=task_list_i,
        make_loader_fn=make_loader_fn_i,
        make_model_fn=make_model_fn_i,
        tfm=tfm_i,
        cfg=cfg_i,
        preset_name=PRESET,
        out_root=ROOT / "outputs",
        verbose=True,
    )
    runs_out.append(out_dir)

print("\nHecho:", [str(p) for p in runs_out])



=== RUN: preset=rate_t16_rgb | method=naive | seed=42 | enc=rate | kwargs={} ===

--- Tarea 1/2: circuito1 | preset=rate_t16_rgb | method=naive | B=64 T=16 AMP=True | enc=rate ---


[TRAIN it/s] epoch 1/2: 5.2 it/s  (391 iters en 74.95s)
[TRAIN it/s] epoch 2/2: 5.0 it/s  (391 iters en 77.54s)

--- Tarea 2/2: circuito2 | preset=rate_t16_rgb | method=naive | B=64 T=16 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 5.2 it/s  (170 iters en 32.55s)
[TRAIN it/s] epoch 2/2: 5.4 it/s  (170 iters en 31.61s)

=== RUN: preset=rate_t16_rgb | method=ewc | seed=42 | enc=rate | kwargs={'lam': 700000000.0, 'fisher_batches': 800} ===

--- Tarea 1/2: circuito1 | preset=rate_t16_rgb | method=ewc_lam_7e+08 | B=64 T=16 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 5.4 it/s  (391 iters en 71.86s)
[TRAIN it/s] epoch 2/2: 5.6 it/s  (391 iters en 70.23s)

--- Tarea 2/2: circuito2 | preset=rate_t16_rgb | method=ewc_lam_7e+08 | B=64 T=16 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 5.0 it/s  (170 iters en 34.12s)
[TRAIN it/s] epoch 2/2: 5.0 it/s  (170 iters en 34.04s)

=== RUN: preset=rate_t16_rgb | method=rehearsal | seed=42 | enc=rate | kwargs={'buffer_size': 5000, 'replay_ratio': 0

<a id="sec-08"></a>

## 8) Barrido de combinaciones (opcional)

Driver genérico para explorar:

- `presets × seeds × encoders × métodos`.

Útil para estudios más amplios (coste alto).  
Asegura H5 compatibles si usas modo offline; controla carga (workers/prefetch) si la GPU va justa.

[↑ Volver al índice](#toc)

In [ ]:
# =============================================================================
# Driver de ejecución: barrido de combinaciones (opcional)
# =============================================================================
from copy import deepcopy
from src.runner import run_continual
from src.utils import load_preset, build_task_list_for, build_components_for

PRESETS   = [PRESET]  # añade "std", "accurate" si quieres
SEEDS     = [CFG["data"]["seed"], 43]
ENCODERS  = [CFG["data"]["encoder"]]
METHODS   = [
    ("naive", {}),
    ("ewc", {"lam": 1e9, "fisher_batches": 600}),
    # ("rehearsal", {"buffer_size": 5000, "replay_ratio": 0.2}),
    # ("rehearsal+ewc", {"buffer_size": 5000, "replay_ratio": 0.2, "lam": 7e8, "fisher_batches": 600}),
]

for preset_i in PRESETS:
    CFG_i = load_preset(ROOT / "configs" / "presets.yaml", preset_i)
    for seed_i in SEEDS:
        for enc_i in ENCODERS:
            for method_name, method_params in METHODS:
                cfg_i2 = deepcopy(CFG_i)
                cfg_i2["data"]["seed"] = seed_i
                cfg_i2["data"]["encoder"] = enc_i
                cfg_i2["continual"]["method"] = method_name
                cfg_i2["continual"]["params"] = method_params
                if "rehearsal" in method_name:
                    cfg_i2["data"]["persistent_workers"] = False

                tfm_i, make_loader_fn_i, make_model_fn_i = build_components_for(cfg_i2, ROOT)
                task_list_i, tasks_file_i = build_task_list_for(cfg_i2, ROOT)

                print(f"\n=== RUN: preset={preset_i} | method={method_name} | seed={seed_i} "
                      f"| enc={enc_i} | kwargs={method_params} ===")
                out_path, _ = run_continual(
                    task_list=task_list_i,
                    make_loader_fn=make_loader_fn_i,
                    make_model_fn=make_model_fn_i,
                    tfm=tfm_i,
                    cfg=cfg_i2,
                    preset_name=preset_i,
                    out_root=ROOT / "outputs",
                    verbose=True,
                )
                print("OK:", out_path)



=== RUN: preset=fast | method=naive | seed=42 | enc=rate | kwargs={} ===

--- Tarea 1/2: circuito1 | preset=fast | method=naive | B=64 T=10 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 10.9 it/s  (391 iters en 35.74s)
[TRAIN it/s] epoch 2/2: 12.6 it/s  (391 iters en 30.93s)

--- Tarea 2/2: circuito2 | preset=fast | method=naive | B=64 T=10 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 11.9 it/s  (170 iters en 14.27s)
[TRAIN it/s] epoch 2/2: 12.7 it/s  (170 iters en 13.41s)
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_naive_rate_model-PilotNetSNN_66x200_gray_seed_42

=== RUN: preset=fast | method=ewc | seed=42 | enc=rate | kwargs={'lam': 1000000000.0, 'fisher_batches': 600} ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_1e+09 | B=64 T=10 AMP=True | enc=rate ---
[TRAIN it/s] epoch 1/2: 12.5 it/s  (391 iters en 31.38s)
[TRAIN it/s] epoch 2/2: 12.9 it/s  (391 iters en 30.36s)

--- Tarea 2/2: circuito2 | preset=fast | method=ewc_lam_1e+09 | B=64 T=10 AMP=True

<a id="sec-09"></a>

## 9) Resumen completo: inventario → parseo → agregados → tabla

- **Inventario** de runs en `outputs/continual_*`.  
- **Parseo** del nombre de las carpetas para extraer `preset`, `método`, `λ`, `encoder`, `seed`, `modelo`.  
- **Cálculo de olvido**: diferencia absoluta y relativa de T1 tras T2.  
- **Agregados** por grupo (media/σ/n) y export a `outputs/summary/continual_summary_agg.csv`.  
- **Tabla formateada** con métricas clave y desviaciones.

> Si no aparece nada, revisa que existan `continual_results.json` en las carpetas de salida.

[↑ Volver al índice](#toc)


In [ ]:
# === Resumen de runs (usa utilidades comunes) ===
from pathlib import Path
from src.utils_exp import build_runs_df, aggregate_and_show

outputs_root = ROOT / "outputs"

# (Opcional) inventario rápido en disco
print("Inventario de runs en:", outputs_root)
for p in sorted(outputs_root.glob("continual_*")):
    print(" -", p.name, "| results.json:", (p / "continual_results.json").exists())

df = build_runs_df(outputs_root)
print(f"runs en resumen: {len(df)}")
_ = aggregate_and_show(df, outputs_root)  # también guarda CSV en outputs/summary/


Inventario de runs en: /home/cesar/proyectos/TFM_SNN/outputs
 - continual_fast_ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_43 | results.json: True
 - continual_fast_ewc_lam_2e+09_lam_2e+09_ewc_hpo_t4_lam_1.8e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_3e+08_lam_3e+08_ewc_hpo_t7_lam_3.3e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_4e+08_lam_4e+08_ewc_hpo_t2_lam_4.5e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_4e+08_lam_4e+08_ewc_hpo_t6_lam_3.6e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_5e+08_lam_5e+08_ewc_hpo_t0_lam_4.8e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_5e+08_lam_5e+08_rate_model-PilotNetSNN_66x200_gray_seed_

,exp,preset,method,lambda,encoder,model,seed,c1_name,c2_name,c1_mae,c1_after_c2_mae,c1_forgetting_mae_abs,c1_forgetting_mae_rel_%,c2_mae,lambda_num
0,continual_fast_ewc_lam_5e+08_lam_5e+08_rate_mo...,fast,ewc,5e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171645,0.000356,0.207714,0.223225,5.000000e+08
1,continual_fast_ewc_lam_7e+08_lam_7e+08_rate_mo...,fast,ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171568,0.000278,0.162523,0.223178,7.000000e+08
2,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171298,0.000008,0.004711,0.223016,1.000000e+09
3,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,43,circuito1,circuito2,0.171586,0.171580,-0.000005,-0.003145,0.223185,1.000000e+09
4,continual_fast_ewc_lam_5e+08_lam_5e+08_ewc_hpo...,fast,ewc_ewc_hpo_t0_lam_4.8e+08,5e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171713,0.171058,-0.000655,-0.381311,0.222880,5.000000e+08
5,continual_fast_ewc_lam_7e+08_lam_7e+08_ewc_hpo...,fast,ewc_ewc_hpo_t1_lam_7.1e+08,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171485,0.000196,0.114275,0.223129,7.000000e+08
6,continual_fast_ewc_lam_4e+08_lam_4e+08_ewc_hpo...,fast,ewc_ewc_hpo_t2_lam_4.5e+08,4e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171640,0.000351,0.204753,0.223222,4.000000e+08
7,continual_fast_ewc_lam_6e+08_lam_6e+08_ewc_hpo...,fast,ewc_ewc_hpo_t3_lam_6.4e+08,6e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171794,0.000504,0.294161,0.223314,6.000000e+08
8,continual_fast_ewc_lam_2e+09_lam_2e+09_ewc_hpo...,fast,ewc_ewc_hpo_t4_lam_1.8e+09,2e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171313,0.000023,0.013333,0.223025,2.000000e+09
9,continual_fast_ewc_lam_8e+08_lam_8e+08_ewc_hpo...,fast,ewc_ewc_hpo_t5_lam_8.1e+08,8e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171290,0.171527,0.000237,0.138447,0.223153,8.000000e+08


Guardado: /home/cesar/proyectos/TFM_SNN/outputs/summary/continual_summary_agg.csv


,preset,método,codificador,λ,MAE Tarea1 (media),Olvido T1 (%) (media),MAE Tarea2 (media),MAE Tarea1 (σ),Olvido T1 (%) (σ),MAE Tarea2 (σ),n (semillas)
0,fast,ewc,rate,5e+08,0.1713,0.2077,0.2232,,,,1
1,fast,ewc,rate,7e+08,0.1713,0.1625,0.2232,,,,1
2,fast,ewc,rate,1e+09,0.1714,0.0008,0.2231,0.0002,0.0056,0.0001,2
3,fast,ewc_ewc_hpo_t0_lam_4.8e+08,rate,5e+08,0.1717,-0.3813,0.2229,,,,1
4,fast,ewc_ewc_hpo_t1_lam_7.1e+08,rate,7e+08,0.1713,0.1143,0.2231,,,,1
5,fast,ewc_ewc_hpo_t2_lam_4.5e+08,rate,4e+08,0.1713,0.2048,0.2232,,,,1
6,fast,ewc_ewc_hpo_t3_lam_6.4e+08,rate,6e+08,0.1713,0.2942,0.2233,,,,1
7,fast,ewc_ewc_hpo_t4_lam_1.8e+09,rate,2e+09,0.1713,0.0133,0.2230,,,,1
8,fast,ewc_ewc_hpo_t5_lam_8.1e+08,rate,8e+08,0.1713,0.1384,0.2232,,,,1
9,fast,ewc_ewc_hpo_t6_lam_3.6e+08,rate,4e+08,0.1713,0.4956,0.2235,,,,1
